# Static Quantization example with `optimum` for `distilbert`

In [ ]:
!pip install -r requirements.txt
!pip install git+https://github.com/huggingface/optimum.git@main "onnx" "onnxruntime>=1.9.0" "datasets>=1.2.1" "protobuf==3.20.1"
!pip install evaluate sklearn

## Configurations

lets define our `model_id` and `dataset`, will be used to statically quantize the model

In [1]:
from pathlib import Path

model_id="mrm8488/distilroberta-finetuned-banking77"
dataset_id="banking77"
onnx_path = Path("onnx")
task="sequence-classification"

## Converting the model to onnx

In [2]:
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import AutoTokenizer

# load vanilla transformers and convert to onnx
model = ORTModelForSequenceClassification.from_pretrained(model_id, from_transformers=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# save onnx checkpoint and tokenizer
model.save_pretrained(onnx_path)
tokenizer.save_pretrained(onnx_path)

/home/ubuntu/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 6.59k/6.59k [00:00<00:00, 6.05MB/s]


('onnx/tokenizer_config.json',
 'onnx/special_tokens_map.json',
 'onnx/vocab.json',
 'onnx/merges.txt',
 'onnx/added_tokens.json',
 'onnx/tokenizer.json')

## Configuring static quantization

In [3]:
from optimum.onnxruntime import ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig

# create ORTQuantizer and define quantization configuration
quantizer = ORTQuantizer.from_pretrained(model_id, feature=task)
qconfig = AutoQuantizationConfig.avx512_vnni(is_static=True, per_channel=False)

## Running Calibration of quantization ranges

In [ ]:
%%capture
import os
from functools import partial
from optimum.onnxruntime.configuration import AutoCalibrationConfig

def preprocess_fn(ex, tokenizer):
    return tokenizer(ex["text"])

# Create the calibration dataset
calibration_dataset = quantizer.get_calibration_dataset(
    dataset_id,
    preprocess_function=partial(preprocess_fn, tokenizer=quantizer.tokenizer),
    num_samples=200,
    dataset_split="train",
)


# Create the calibration configuration containing the parameters related to calibration.
calibration_config = AutoCalibrationConfig.minmax(calibration_dataset)
# Perform the calibration step: computes the activations quantization ranges
ranges = quantizer.fit(
    dataset=calibration_dataset,
    calibration_config=calibration_config,
    onnx_model_path=onnx_path / "model.onnx",
    operators_to_quantize=qconfig.operators_to_quantize,
)
# remove temp augmented model again
os.remove("augmented_model.onnx")


## Running quantization with calibrated ranges

In [5]:
from optimum.onnxruntime.preprocessors import QuantizationPreprocessor
from optimum.onnxruntime.preprocessors.passes import (
    ExcludeGeLUNodes,
    ExcludeLayerNormNodes,
    ExcludeNodeAfter,
    ExcludeNodeFollowedBy,
)

# Create a quantization preprocessor to determine the nodes to exclude
quantization_preprocessor = QuantizationPreprocessor()

# Exclude the nodes constituting LayerNorm
quantization_preprocessor.register_pass(ExcludeLayerNormNodes())
# Exclude the nodes constituting GELU
quantization_preprocessor.register_pass(ExcludeGeLUNodes())
# Exclude the residual connection Add nodes
quantization_preprocessor.register_pass(ExcludeNodeAfter("Add", "Add"))
# Exclude the Add nodes following the Gather operator
quantization_preprocessor.register_pass(ExcludeNodeAfter("Gather", "Add"))
# Exclude the Add nodes followed by the Softmax operator
quantization_preprocessor.register_pass(ExcludeNodeFollowedBy("Add", "Softmax"))

In [6]:
# Quantize the same way we did for dynamic quantization!
quantizer.export(
    onnx_model_path=onnx_path / "model.onnx",
    onnx_quantized_model_output_path=onnx_path / "model-quantized.onnx",
    calibration_tensors_range=ranges,
    quantization_config=qconfig,
    # preprocessor=quantization_preprocessor,
)

PosixPath('onnx/model-quantized.onnx')

In [7]:
import os

# get model file size
size = os.path.getsize(onnx_path / "model.onnx")/(1024*1024)
quantized_model = os.path.getsize(onnx_path / "model-quantized.onnx")/(1024*1024)

print(f"Model file size: {size:.2f} MB")
print(f"Quantized Model file size: {quantized_model:.2f} MB")

Model file size: 313.54 MB
Quantized Model file size: 192.03 MB


## Test quantized model

In [8]:
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import pipeline, AutoTokenizer

model = ORTModelForSequenceClassification.from_pretrained(onnx_path,file_name="model-quantized.onnx")

clx = pipeline("text-classification",model=model, tokenizer=quantizer.tokenizer)

In [9]:
clx("Please transfer 500€")

[{'label': 'top_up_by_bank_transfer_charge', 'score': 0.13707876205444336}]

## Evaluate

In [10]:
from transformers import pipeline
import evaluate
from datasets import load_dataset


test_dataset= load_dataset(dataset_id,split="test")
metric = evaluate.load("accuracy")

def evaluate_model(pipe,dataset,metric,text_column="text",label_column="label"):
    def eval(sample):
        onnx = pipe(sample[text_column])
        return {"pred": pipe.model.config.label2id[onnx[0][label_column]] ,"ref": sample[label_column]}

    res = test_dataset.map(eval)
    return metric.compute(references=res["ref"], predictions=res["pred"])

Using custom data configuration default
Reusing dataset banking77 (/home/ubuntu/.cache/huggingface/datasets/banking77/default/1.1.0/aec0289529599d4572d76ab00c8944cb84f88410ad0c9e7da26189d31f62a55b)
Couldn't find a directory or a metric named 'accuracy' in this version. It was picked from the master branch on github instead.
100%|██████████| 3080/3080 [00:59<00:00, 52.15ex/s]


{'accuracy': 0.8961038961038961}

original model

In [11]:
pipe = pipeline("text-classification",model=model_id)

evaluate_model(pipe,test_dataset,metric)



100%|██████████| 3080/3080 [00:59<00:00, 51.38ex/s]


{'accuracy': 0.8961038961038961}

quantized model

In [12]:
evaluate_model(clx,test_dataset,metric)

Parameter 'function'=<function evaluate_model.<locals>.eval at 0x7f0aba439a60> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████| 3080/3080 [00:15<00:00, 197.19ex/s]


{'accuracy': 0.8662337662337662}

## Optimize the quantized model

In [13]:
from optimum.onnxruntime import ORTOptimizer
from optimum.onnxruntime.configuration import OptimizationConfig

# create ORTOptimizer and define optimization configuration
optimizer = ORTOptimizer.from_pretrained(model_id, feature=task)
optimization_config = OptimizationConfig(optimization_level=99) # enable all optimizations

# apply the optimization configuration to the model
optimizer.export(
    onnx_model_path=onnx_path / "model-quantized.onnx",
    onnx_optimized_model_output_path=onnx_path / "model-optimized.onnx",
    optimization_config=optimization_config,
)

2022-06-02 13:31:32.343331561 [W:onnxruntime:, inference_session.cc:1546 Initialize] Serializing optimized model with Graph Optimization level greater than ORT_ENABLE_EXTENDED and the NchwcTransformer enabled. The generated model may contain hardware specific optimizations, and should only be used in the same environment the model was optimized in.
Model producer not matched: Expect pytorch, Got onnx.quantize 0.1.0. Please specify correct --model_type parameter.
Attention not fused


PosixPath('onnx/model-optimized.onnx')

In [14]:
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import pipeline, AutoTokenizer

model = ORTModelForSequenceClassification.from_pretrained(onnx_path,file_name="model-optimized.onnx")

optimized_clx = pipeline("text-classification",model=model, tokenizer=quantizer.tokenizer)

But first, let’s extend our context and question to a more meaningful sequence length of 128.

In [15]:
payload="What do I need to do to get my new card which I have requested 2 weeks ago?"

In [16]:
payload="Hello my name is Philipp. I am getting in touch with you because i didn't get a response from you. What do I need to do to get my new card which I have requested 2 weeks ago? Please help me and answer this email in the next 7 days. Best regards and have a nice weekend "*2
len(tokenizer(payload)["input_ids"])

127

In [17]:
from time import perf_counter
import numpy as np 

def measure_latency(pipe):
    latencies = []
    # warm up
    for _ in range(10):
        _ = pipe(payload)
    # Timed run
    for _ in range(300):
        start_time = perf_counter()
        _ =  pipe(payload)
        latency = perf_counter() - start_time
        latencies.append(latency)
    # Compute run statistics
    time_avg_ms = 1000 * np.mean(latencies)
    time_std_ms = 1000 * np.std(latencies)
    time_p95_ms = 1000 * np.percentile(latencies,95)
    return f"P95 latency (ms) - {time_p95_ms}; Average latency (ms) - {time_avg_ms:.2f} +\- {time_std_ms:.2f};", time_avg_ms


vanilla_model=measure_latency(pipe)
quantized_model=measure_latency(clx)
optimized_model=measure_latency(optimized_clx)

print(f"Vanilla model: {vanilla_model[0]}")
print(f"Quantized model: {quantized_model[0]}")
print(f"Optimized model: {optimized_model[0]}")
print(f"Improvement through quantization: {round(vanilla_model[1]/quantized_model[1],2)}x")
print(f"Improvement through optimization: {round(vanilla_model[1]/optimized_model[1],2)}x")

Vanilla model: P95 latency (ms) - 66.20852575069875; Average latency (ms) - 56.69 +\- 9.12;
Quantized model: P95 latency (ms) - 24.14308940024057; Average latency (ms) - 23.89 +\- 0.34;
Optimized model: P95 latency (ms) - 23.938274949250626; Average latency (ms) - 23.73 +\- 0.65;
Improvement through quantization: 2.37x
Improvement through optimization: 2.39x
